In [6]:
# Passo 0 - verificar qual a pasta de trabalho "flagada".
import os
os.getcwd()

'C:\\Users\\charl'

# Passo 1. Criar a Pasta de Templates (HTML)
 Pasta onde os arquivos/layouts HTML serão armazenados. O Flask procura arquivos de template na pasta chamada "templates" por padrão.

In [12]:
os.makedirs('templates', exist_ok=True)


# 2. Criar os Arquivos HTML Dentro da Pasta Templates


### 2.1 Criando o template index 

In [21]:
with open('templates/index.html', 'w') as f:
    f.write('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Controle de Estoque</title>
    </head>
    <body>
        <h1>Controle de Estoque</h1>
        <p>Estoque Atual:</p>
        <!-- conteúdo será adicionado dinamicamente pelo Flask -->
    </body>
    </html>
    ''')


### 2.2 Criando o template "Reserva Confirmada"

In [30]:
with open('templates/reserva_confirmada.html', 'w') as f:
    f.write('''
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Reserva Confirmada</title>
    </head>
    <body>
        <h1>Parabéns, sua reserva foi efetuada!</h1>
        <p>Atenção: você tem até 48h úteis para retirar seu pedido.</p>
        <p>Você está locando os seguintes itens:</p>
        <!-- lista de itens será renderizada pelo Flask -->
    </body>
    </html>
    ''')


In [34]:
# 2.3 - Para confirmar que os arquivos foram criados, use esse comando para listar:
os.listdir('templates')


['index.html', 'reserva_confirmada.html']

# 3. Criando o aplicativo com o flask
Dado que já foram criados os templates no passo anterior, o Flask utilizará os arquivos HTML para renderizar as páginas.

In [39]:
# Se não tiver instalado, rodar os codigos a seguir
#!pip install Flask
#!pip install flask_sqlalchemy
#!pip install flask_ngrok

In [ ]:
from flask import Flask, render_template, request, redirect, url_for
from flask_ngrok import run_with_ngrok  # Importa o ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Inicia o ngrok junto com o Flask

# Estoque inicial
estoque = {
    'netbooks': 160,
    'tablets': 80,
    'chromebooks': 40,
    'salas_audiovisual': 3
}

# Reservas
reservas = {
    'netbooks': 0,
    'tablets': 0,
    'chromebooks': 0,
    'salas_audiovisual': 0
}

@app.route('/')
def index():
    # Calcula o estoque atual com base nas reservas
    estoque_atual = {item: estoque[item] - reservas[item] for item in estoque}
    return render_template('index.html', estoque_atual=estoque_atual)

@app.route('/solicitar', methods=['GET', 'POST'])
def solicitar():
    if request.method == 'POST':
        item = request.form['item']
        quantidade = int(request.form['quantidade'])
        prioridade = 'prioridade' in request.form
        
        # Verifica disponibilidade
        if estoque[item] - reservas[item] >= quantidade:
            reservas[item] += quantidade
            return redirect(url_for('reserva_confirmada', item=item, quantidade=quantidade, prioridade=prioridade))
        else:
            return "Estoque insuficiente!", 400
    
    return render_template('solicitar.html')

@app.route('/reserva_confirmada')
def reserva_confirmada():
    item = request.args.get('item')
    quantidade = request.args.get('quantidade')
    prioridade = request.args.get('prioridade', False)
    return render_template('reserva_confirmada.html', item=item, quantidade=quantidade, prioridade=prioridade)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
Exception in thread Thread-7:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connection.py", line 203, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 85, in create_connection
    raise err
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 791, in urlopen
    response = self._make_request(
               ^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-pac

### 3.1 Rodar o Flask e Ngrok
Depois de rodar o código acima, o ngrok vai gerar um link público que permitirá acessar o seu servidor Flask remotamente. 

### OBS: Se rodar adequadamente o aplicativo com o layout HTML, utilize o link disponibilizado. <br> Caso não rode, pode ser algum problema de firewall em razão do uso da porta 4040 ou do ngrok.

# 4. (EXTRA) Caso o link não funcione

### 4.1 Copie o código completo do Flask (passo 3) para um arquivo Python (.py) separado
Chamei de controle_estoque.py e salvei na pasta de trabalho Templates. 


### 4.2 Rodar o Flask localmente
 Abra o terminal ou prompt de comando (atenção: fora do Jupyter Notebook).

Navegue até a pasta onde você salvou o arquivo controle_estoque.py. 
Use o comando cd para navegar no diretório no terminal. Segue exemplo:
<br> Suponde que tenha sido salvo em **C:\Users\charl\templates**, no terminal insira o comando
**cd C:\Users\charl\templates**
<br> Feito isso, rode o codigo Flask com o seguinte comando.
<br> **python controle_estoque.py**

Resultado esperado: Isso deve iniciar o servidor Flask localmente , exibindo uma mensagem como: <br> * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
